In [9]:
# please use Python 3 if possible

#!pip install bayesian-optimization
!pip install git+https://github.com/slremy/netsapi --user --upgrade

from bayes_opt import BayesianOptimization
from bayes_opt.util import UtilityFunction
%matplotlib inline
#challenge import
from netsapi.challenge import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import mlab
from matplotlib import gridspec
%matplotlib inline
from sys import exit, exc_info, argv
from multiprocessing import Pool, current_process
import random as rand
import json
import requests
import numpy as np
import pandas as pd
import statistics
from IPython.display import clear_output
from contextlib import contextmanager
import sys, os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

class CustomAgent():
    
    def __init__(self, environment):
        
        #Hyperparameters
        self.env = environment
        
        #action space:
        self.x_start = 0.0
        self.x_end = 1.0
        
        self.bo = BayesianOptimization(f=self.target,
                                       pbounds={'Y1x': (self.x_start, self.x_end), 'Y1y': (self.x_start, self.x_end), 'Y2x': (self.x_start, self.x_end), 'Y2y': (self.x_start, self.x_end), 'Y3x': (self.x_start, self.x_end), 'Y3y': (self.x_start, self.x_end), 'Y4x': (self.x_start, self.x_end), 'Y4y': (self.x_start, self.x_end), 'Y5x': (self.x_start, self.x_end), 'Y5y': (self.x_start, self.x_end)},
                                       verbose=0)
        normalize =  None
        
    def target1(self, Y1x,Y1y,Y2x,Y2y,Y3x,Y3y,Y4x,Y4y,Y5x,Y5y):
        Y1x = np.asscalar(Y1x)
        Y1y = np.asscalar(Y1y)

        Y2x = np.asscalar(Y2x)
        Y2y = np.asscalar(Y2y)

        Y3x = np.asscalar(Y3x)
        Y3y = np.asscalar(Y3y)

        Y4x = np.asscalar(Y4x)
        Y4y = np.asscalar(Y4y)

        Y5x = np.asscalar(Y5x)
        Y5y = np.asscalar(Y5y)

        policy = {1:[Y1x, Y1y], 2:[Y2x, Y2y], 3:[Y3x, Y3y], 4:[Y4x, Y4y], 5:[Y5x, Y5y]}

        self.env.reset()
        reward_AllPolicy = self.env.evaluatePolicy(policy)

        return reward_AllPolicy/self.normalize    

    def target(self, Y1x,Y1y,Y2x,Y2y,Y3x,Y3y,Y4x,Y4y,Y5x,Y5y):
        if type(Y1x) is np.ndarray:
            result = []
            for a,b,c,d,e,f,g,h,i,j in zip(Y1x,Y1y,Y2x,Y2y,Y3x,Y3y,Y4x,Y4y,Y5x,Y5y):
                reward = target1(a,b,c,d,e,f,g,h,i,j)
                result.append( reward )
            return result
        else:
            return self.target1(Y1x,Y1y,Y2x,Y2y,Y3x,Y3y,Y4x,Y4y,Y5x,Y5y)    
        



    def generate(self):
        #self.bo.maximize(init_points=5, n_iter=0, acq='ucb', kappa=10)
        next_point_to_probes = []
        firstRewards = []
        for _ in range(5):
            Y1x = rand.random()
            Y1y = rand.random()

            Y2x = rand.random()
            Y2y = rand.random()

            Y3x = rand.random()
            Y3y = rand.random()

            Y4x = rand.random()
            Y4y = rand.random()

            Y5x = rand.random()
            Y5y = rand.random()
            policy = {1:[Y1x,Y1y],2:[Y2x,Y2y],3:[Y3x,Y3y],4:[Y4x,Y4y],5:[Y5x,Y5y]}
            next_point_to_probe = {'Y1x': Y1x, 'Y1y': Y1y, 'Y2x': Y2x, 'Y2y': Y2y, 'Y3x':Y3x, 'Y3y': Y3y, 'Y4x': Y4x, 'Y4y': Y4y, 'Y5x':Y5x, 'Y5y':Y5y}
            next_point_to_probes.append(next_point_to_probe)
            
            self.env.reset()
            reward_AllPolicy = self.env.evaluatePolicy(policy)
            firstRewards.append(reward_AllPolicy)
        
        
            
            
        tmpRewards = np.array(firstRewards)
        self.normalize = np.mean(np.abs(tmpRewards))    
        print("normalize ", self.normalize)    
        
        for p,r in zip(next_point_to_probes,firstRewards):
            self.bo.register(
                params=p,
                target= r / self.normalize,
                )
        
        self.bo.maximize(init_points=0, n_iter=0, acq='ucb', kappa=10)
        for i in range(10):
            self.bo.maximize(init_points=0, n_iter=1, acq='ucb', kappa=9)
        for i in range(3):
            self.bo.maximize(init_points=0, n_iter=1, acq='ucb', kappa=2.5)
        self.bo.maximize(init_points=0, n_iter=1, acq='ucb', kappa=1)
        self.bo.maximize(init_points=0, n_iter=1, acq='ucb', kappa=0)

        lista = self.bo.res[:]

        lista.sort(key=lambda x: x['target'], reverse=True)
        #print("predicted: ", lista[0]['target']*90.0)
        
        policy= lista[0]['params']
        best_policy = {1:[policy['Y1x'],policy['Y1y']], 2:[policy['Y2x'],policy['Y2y']], 3:[policy['Y3x'] ,policy['Y3y']], 4:[policy['Y4x']     , policy['Y4y']], 5:[policy['Y5x'] , policy['Y5y']]}
        #print(best_policy)
        
        best_reward = self.env.evaluatePolicy(best_policy)
        #print("found ", reward_AllPolicy)
        
        print(best_policy, best_reward)
        
        return best_policy, best_reward
    
    
    
EvaluateChallengeSubmission(ChallengeSeqDecEnvironment, CustomAgent, "example.csv") #How scoring files generated by participants should *not* be included in submission.py file

  Cloning https://github.com/slremy/netsapi to c:\users\karim\appdata\local\temp\pip-req-build-2b9943nq
  Stored in directory: C:\Users\Karim\AppData\Local\Temp\pip-ephem-wheel-cache-nju1to5r\wheels\9e\73\c9\86a9cc2460e11b3ce5b0a5ebd2d9d332a68afe0941659967fa
Successfully built netsapi
  Found existing installation: netsapi 1.2
    Uninstalling netsapi-1.2:
      Successfully uninstalled netsapi-1.2
105  Evaluations Remaining
100  Evaluations Remaining
95  Evaluations Remaining
90  Evaluations Remaining
85  Evaluations Remaining
normalize  39.60080686223995
80  Evaluations Remaining
75  Evaluations Remaining
70  Evaluations Remaining
65  Evaluations Remaining
60  Evaluations Remaining
55  Evaluations Remaining
50  Evaluations Remaining
45  Evaluations Remaining
40  Evaluations Remaining
35  Evaluations Remaining
30  Evaluations Remaining
25  Evaluations Remaining
20  Evaluations Remaining
15  Evaluations Remaining
10  Evaluations Remaining
5  Evaluations Remaining
{1: [0.568017959916183